In [1]:
#การเชื่อม google drive ซึ่งหากไม่ได้รันบน Google Colab ไม่ต้องรัน cell นี้
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# COVID-19 Classification model

## กำหนด pathและโมเดลที่จะใช้ใน cell ด้านล่างนี้

In [2]:
#กำหนดpathของโฟลเดอร์ที่เก็บข้อมูลภาพที่ต้องการทำนาย
data_path = '/content/drive/MyDrive/COVID-19_Classification/sampleData'
#กำหนดโมเดลที่ต้องการใช้ [CNN หรือ ResNet50]
model_type = 'CNN'
#กำหนดpathของไฟล์weightของโมเดลที่จะใช้
model_path = '/content/drive/MyDrive/COVID-19_Classification/CNN_Model_noAugment.hdf5'

In [3]:
from contextlib import AsyncExitStack
import os
import numpy as np
import cv2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Dropout, Conv2D, BatchNormalization
from keras.layers import MaxPooling2D, Flatten
from keras.models import Model
from keras.models import Sequential

def build_resnet50():
    resnet50 = ResNet50(weights='imagenet', include_top=False)
    for each_layer in resnet50.layers:
        each_layer.trainable=False
    input = Input(shape=(64, 64, 3))
    x = Conv2D(3, (3, 3), padding='same')(input)
    x = resnet50(x)
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    output = Dense(1,activation = 'sigmoid', name='root')(x)
    model = Model(input,output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
    model.summary()
    return model

def build_CNN():
    model = Sequential()
    model.add(Conv2D(32, (3, 3),padding='same', input_shape = (64, 64, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Conv2D(64, (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Conv2D(128, (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Flatten())
    model.add(Dense(units = 64, activation = 'relu'))
    model.add(Dense(units = 1, activation = 'sigmoid'))
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    model.summary()
    return model


def build_model(model_type):
    files = os.listdir(data_path)
    dataset = np.zeros((len(files),64,64, 3))
    for i,file in enumerate(files):
        image = cv2.imread(os.path.join(data_path, file))
        if image is not None:
            dataset[i] = image
    X_images = dataset / 255.
    md = model_type.lower()
    if md == 'cnn':
        model = build_CNN()
        model.load_weights(model_path)
        predict = model.predict(X_images)
        y_predict = []
        for i in predict:
            if i >= 0.5:
                y_predict.append('COVID')
            else:
                y_predict.append('Normal')
        ans = list(zip(files,y_predict))
        return ans
    elif md == 'resnet50':
        model = build_resnet50()
        model.load_weights(model_path)
        predict = model.predict(X_images)
        y_predict = []
        for i in predict:
            if i >= 0.5:
                y_predict.append('COVID')
            else:
                y_predict.append('Normal')
        ans = list(zip(files,y_predict))
        return ans
    else:
        print('Please fill model_type value in first cell. [CNN or ResNet50]')


In [4]:
predict = build_model(model_type)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 15, 15, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 13, 13, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 128)        0

## ผลการจำแนก

In [5]:
predict

[('testNormal1.png', 'Normal'),
 ('testNormal2.png', 'Normal'),
 ('testNormal6.png', 'Normal'),
 ('testNormal7.png', 'Normal'),
 ('testNormal8.png', 'Normal'),
 ('testNormal3.png', 'Normal'),
 ('testNormal5.png', 'Normal'),
 ('testNormal4.png', 'Normal'),
 ('testNormal9.png', 'Normal'),
 ('testNormal10.png', 'Normal'),
 ('testNormal14.png', 'Normal'),
 ('testNormal15.png', 'Normal'),
 ('testNormal12.png', 'Normal'),
 ('testNormal11.png', 'Normal'),
 ('testNormal16.png', 'Normal'),
 ('testNormal13.png', 'Normal'),
 ('testNormal18.png', 'Normal'),
 ('testNormal17.png', 'Normal'),
 ('testCOVID4.png', 'COVID'),
 ('testCOVID1.png', 'COVID'),
 ('testCOVID5.png', 'COVID'),
 ('testCOVID3.png', 'COVID'),
 ('testCOVID2.png', 'COVID'),
 ('testCOVID6.png', 'COVID'),
 ('testCOVID10.png', 'COVID'),
 ('testCOVID7.png', 'COVID'),
 ('testCOVID9.png', 'COVID'),
 ('testCOVID11.png', 'Normal'),
 ('testCOVID8.png', 'COVID'),
 ('testCOVID12.png', 'COVID'),
 ('testCOVID17.png', 'COVID'),
 ('testCOVID13.png',